In [ ]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
# data/reloading_data - CATRIDGES.csv
cartridges = pd.read_csv('data/reloading_data - CATRIDGES.csv')
print(cartridges)

In [ ]:
cartridges_to_process = cartridges.title.to_list()
print(cartridges_to_process)

In [ ]:
data = None
for c in cartridges_to_process:
    filename = f'data/reloading_data - %s.csv' % c
    if not os.path.exists(filename):
        continue
    d = pd.read_csv(filename)
    if data is None:
        data = d
    else:
        data = pd.concat([data, d])
print(data.head())

In [ ]:
d

In [ ]:
data

In [ ]:
[ [10, 20] ] * 5